# Parametric Regression  

In this notebook, we attemps to find optimal parameters for the model by using various nonlinear parametric methods

In [1]:
import os
import time
import shutil
import contextlib
import numpy as np
import matplotlib.pyplot as plt


from helper import *
from models import *
from solvers.euler import *
from data_generator import *
from solvers.gn_solver import *
from solvers.rk_solvers import *


print(f"Your machine has {os.cpu_count()} cores.")


Your machine has 8 cores.


## 1 Gauss Newton

First obtain some data as a list of tuples `total`. The first component is the initial conditions of `(201,2)` while the second is of the shape `(1000, 201, 2)`

In [2]:
start = time.time()

total = generateData(model4,
        num_data = 1,
        init_sty = 'random',
        times = (0, 20),
        params = {'no. of prey': 100, 
    'kappa for prey': 0.5, 
    'attraction of prey a': 0, 
    'repulsion of prey b_1': 0.15, 
    'repulsion of pred b_2': 0.55,
    'p1 spotted': 0.8,
    'p2 not spotted':0.4,
    'angle_prey': np.cos(np.pi / 3),
    'angle_pred': np.cos(np.pi / 2.5), 
    'num_neighbours': 5,
    'attraction of pred c': 10, 
    'exponent of dist pred p': 3},
        steps = 1000,
        second_order = False,
        method = 'rk2',
        return_vel = False,
        cores = 1,
        flattened=True)
end = time.time()
print(f"Time taken: {end-start} seconds.")

Trying to use multiprocessing...
Multiprocessing successful.
Time taken: 3.06046199798584 seconds.


We still need a function that takes in parameter vector instead of a dictionary for regression; we take `attraction of prey a`, `repulsion of prey b_1`, `repulsion of pred b_2` as parameters for regression.

In [3]:
def fModel4(x, vec):
    """
    Returns a flattened output.  

    Args:  
        - x: ndarray, initial conditions
        - vec: ndarray, parameters of length 3
    
    """  
    params = {'no. of prey': 100, 
    'kappa for prey': 0.5, 
    'attraction of prey a': vec[0], 
    'repulsion of prey b_1': vec[1], 
    'repulsion of pred b_2': vec[2],
    'p1 spotted': 0.8,
    'p2 not spotted':0.4,
    'angle_prey': np.cos(np.pi / 3),
    'angle_pred': np.cos(np.pi / 2.5), 
    'num_neighbours': 5,
    'attraction of pred c': 10, 
    'exponent of dist pred p': 3}
    # print(params)
    N = 100
    times = (0,20)
    steps = 1000
    start, end = times  
    h = (end - start) / steps
    x = x.reshape(-1, 2) # reshape to coordinate form of initial conditions
    y = rk2(model4, x, h, 2*N+1, times, params, return_vel=False, verbose=False)

    return np.ravel(y)

Feed in one sample and try the `GaussNewtonSolver`. Note it might be better to drop the zero values in the input

In [4]:
x0 = total[0][0]
y0 = total[0][1]
guess = np.random.rand(3)
gnsolver = GaussNewtonSolver(fModel4, max_iter=2, step = 1e-4)
gnsolver.fit(x0, y0, 0.1, guess, 0.01, 0.1)

  0%|          | 0/2 [00:00<?, ?it/s]

Starting trial 0
Computing jacobian
Computing dk
Starting Line Search
Computing loss jacobian
(1, 3)
Computing loss jacobian


 50%|█████     | 1/2 [01:05<01:05, 65.43s/it]

(1, 3)
Linear Search complete
Updating parameters
Starting trial 1
Computing jacobian
Computing dk
Starting Line Search
Computing loss jacobian
(1, 3)
Computing loss jacobian
(1, 3)
Computing loss jacobian
(1, 3)
Computing loss jacobian


100%|██████████| 2/2 [02:56<00:00, 88.04s/it]

(1, 3)
Max iteration reached for line search!
Linear Search complete
Updating parameters


array([0.00469683, 0.3850813 , 0.08859875])